<a href="https://colab.research.google.com/github/rturquier/depythons/blob/main/Rendu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Projet informatique - Python pour le data scientist
##### Jérémie Stym-Popper, Luca Teodorescu, Rémi Turquier
# Les députés de l'opposition s'expriment-il différemment de ceux de la majorité ?

[Titre temporaire, c'est juste pour voir si ça marche]

In [12]:
# Imports
import pandas as pd

## Récupération des données
Les données ont été récupérées par l'intermédiaire de [l'API](https://github.com/regardscitoyens/nosdeputes.fr/blob/master/doc/api.md) mise à disposition par l'association Regards citoyens.

Nous avons d'abord utilisé un module nommé depute_api, que nous avons ensuite complété avec trois fonctions :
    Les fonctions *interventions* et *interventions2* permettent d'entrer le nom d'un député pour obtenir une liste d'interventions (sous forme de liste de str).
    La fonction ...

In [13]:
# ----- Codage de l'API --------------------------------------------------
from operator import itemgetter
import requests
import warnings
import re
import bs4
import unidecode
from urllib import request


with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    from fuzzywuzzy.process import extractBests


__all__ = ['CPCApi']


def memoize(f):
    cache = {}

    def aux(*args, **kargs):
        k = (args, tuple(sorted(kargs.items())))
        if k not in cache:
            cache[k] = f(*args, **kargs)
        return cache[k]
    return aux


class CPCApi(object):
    format = 'json'

    def __init__(self, ptype='depute', legislature=None):
        """
        type: depute or senateur
        legislature: 2007-2012 or None
        """

        assert(ptype in ['depute', 'senateur'])
        assert(legislature in ['2007-2012', '2012-2017', None])
        self.legislature = legislature
        self.ptype = ptype
        self.ptype_plural = ptype + 's'
        self.base_url = 'https://%s.nos%s.fr' % (legislature or 'www', self.ptype_plural)

    def synthese(self, month=None):
        """
        month format: YYYYMM
        """
        if month is None and self.legislature == '2012-2017':
            raise AssertionError('Global Synthesis on legislature does not work, see https://github.com/regardscitoyens/nosdeputes.fr/issues/69')

        if month is None:
            month = 'data'

        url = '%s/synthese/%s/%s' % (self.base_url, month, self.format)

        data = requests.get(url).json()
        return [depute[self.ptype] for depute in data[self.ptype_plural]]

    def parlementaire(self, slug_name):
        url = '%s/%s/%s' % (self.base_url, slug_name, self.format)
        return requests.get(url).json()[self.ptype]

    def picture(self, slug_name, pixels='60'):
        return requests.get(self.picture_url(slug_name, pixels=pixels))

    def picture_url(self, slug_name, pixels='60'):
        return '%s/%s/photo/%s/%s' % (self.base_url, self.ptype, slug_name, pixels)

    def search(self, q, page=1):
        # XXX : the response with json format is not a valid json :'(
        # Temporary return csv raw data
        url = '%s/recherche/%s?page=%s&format=%s' % (self.base_url, q, page, 'csv')
        return requests.get(url).content
    
    
    @memoize
    def parlementaires(self, active=None):
        if active is None:
            url = '%s/%s/%s' % (self.base_url, self.ptype_plural, self.format)
        else:
            url = '%s/%s/enmandat/%s' % (self.base_url, self.ptype_plural, self.format)

        data = requests.get(url).json()
        return [depute[self.ptype] for depute in data[self.ptype_plural]]
    
    def search_parlementaires(self, q, field='nom', limit=5):
        return extractBests(q, self.parlementaires(), processor=lambda x: x[field] if type(x) == dict else x, limit=limit)



    def interventions(self, dep_name, n_sessions=10, start=4850):
        name = self.search_parlementaires(dep_name)[0][0]["nom"]
        dep_intervention = []
        pattern = "(?<=Permalien" + name + ")" + ".*?(?=Voir tous les commentaires)"
        for num_txt in range(start, start + n_sessions):
            url = "https://www.nosdeputes.fr/15/seance/%s" % (str(num_txt))
            source = request.urlopen(url).read()            
            # source.encoding = source.apparent_encoding
            page = bs4.BeautifulSoup(source, "lxml")
            x = re.findall(pattern, page.get_text(), flags=re.S)
            dep_intervention += x

        return dep_intervention
    
    def interventions2(self, dep_name):
        name = self.search_parlementaires(dep_name)[0][0]["nom"]
        name_pattern = re.sub(' ', '+', unidecode.unidecode(name.lower()))
        dep_intervention = []
        url = "https://www.nosdeputes.fr/recherche?object_name=Intervention&tag=parlementaire%3D{0}&sort=1".format(name_pattern)
        source = request.urlopen(url).read()            
        page = bs4.BeautifulSoup(source, "lxml")
        for x in page.find_all('p', {'class' : 'content'}):
            dep_intervention += x

        return dep_intervention
    
    def liste_mots(self, dep_name):
        name = self.search_parlementaires(dep_name)[0][0]["nom"]
        name_pattern = re.sub(' ', '-', unidecode.unidecode(name.lower()))
        mots_dep = []
        url = "https://www.nosdeputes.fr/{0}/tags".format(name_pattern)
        source = request.urlopen(url).read()            
        page = bs4.BeautifulSoup(source, "lxml")
        for x in page.find_all('span', {'class' : 'tag_level_4'}):
            mots_dep.append(re.sub("\n", "", x.get_text()))
            
        return mots_dep


Ensuite, nous avons créé plusieurs DataFrames à l'aide de la fonction interventions 2, avec les fonctions suivantes : 

In [14]:
# Fonctions intermédiaires

def deputies_of_group(group, n_deputies):
    all_names = deputies_df[deputies_df["groupe_sigle"] == group]["nom"]
    return all_names[:n_deputies]


def interventions_of_group(group, n_deputies=15):
    names = deputies_of_group(group, n_deputies)
    print(names)
    interventions = []
    for name in names:
        print(name)
        interventions += [[group, name, api.interventions2(name)]]
    return interventions

# Fonction de stockage des interventions 

def stockintervention(groupe):
    interventions_group = []
    nbdep = deputies_df.groupby('groupe_sigle')['nom'].count()[str(groupe)]
    print(nbdep)
    interventions_group += interventions_of_group(groupe, nbdep)
    interventions_df = pd.DataFrame(
        interventions_group,
        columns=["groupe", "nom", "interventions"]
        )
    
    return interventions_df

Voici un exemple d'utilisation. Pour éviter de perdre du temps ici (la fonction peut mettre du temps à s'exécuter sur les échantillons de grande taille), on exécute la fonction sur un petit parti politique.

In [19]:
api = CPCApi()
deputies_json = api.parlementaires()
deputies_df = pd.json_normalize(deputies_json)

UAI_inter_df = stockintervention('UAI')
UAI_inter_df

4
76           Yves Jégo
80      Franck Riester
289      Maurice Leroy
515    Napole Polutele
Name: nom, dtype: object
Yves Jégo
Franck Riester
Maurice Leroy
Napole Polutele


,groupe,nom,interventions
0,UAI,Yves Jégo,"[Yves Jégo Prochaine séance, ce soir, à vingt..."
1,UAI,Franck Riester,"[ Vous pouvez compter sur le groupe UDI, Agir ..."
2,UAI,Maurice Leroy,[Maurice Leroy – Nous avons examiné la PPE en...
3,UAI,Napole Polutele,"[ Oui, madame la présidente. , Madame la mini..."


## Nettoyage des données
Après une exploration préliminaire, nous avons choisi de nous concentrer sur deux groupes parlementaires. Les groupes LFI et LR nous ont paru ceux les plus susceptibles d'utiliser un discours différent et que nous pourrions distinguer dans notre modèle. 

In [10]:
# Import des données brutes récupérées avec l'API
data_url = "https://raw.githubusercontent.com/rturquier/depythons/main/Stock_csv/gd2_inter.csv"
df_brut = pd.read_csv(data_url)
df_brut.sample(n=5)

,groupe,nom,interventions
55,LR,Jean-Pierre Vigier,"[' Nul ! ', "" Ce n'était pas clair ! "", ' Elle..."
100,LR,Dino Cinieri,"["" La demi-part supplémentaire qui revient de ..."
151,SOC,Valérie Rabault,"["" … rien pour celles et ceux qui ont perdu le..."
1,LFI,Muriel Ressiguier,"["" L'engagement de la procédure accélérée sur ..."
154,SOC,Serge Letchimy,"["" Monsieur le ministre des outre-mer, des plu..."


In [154]:
# Création d'une indicatrice `droite` qui sera la cible de la classification
df_brut = df_brut.\
            assign(droite = df_brut["groupe"] == "LR").\
            sort_values(by = ["droite"], ascending = False)
df_brut.head()

,Unnamed: 0,groupe,nom,interventions,droite
84,84,LR,Gilles Lurton,"["" Monsieur le Premier ministre, vous nous ave...",True
34,34,LR,Éric Ciotti,"["" J'entends votre argumentation. J'ai évoqué ...",True
22,22,LR,Philippe Gosselin,"[' Excellent rapport ! ', "" C'est très inquiét...",True
23,23,LR,Vincent Descoeur,"["" C'est vrai ! "", "" C'est une anomalie ! "", '...",True
86,86,LR,Jean-François Parigi,"[' Restez modeste ! ', ' Manipulation ! ', ' C...",True


In [155]:
# Équilibrage du nombre de députés
def balance_left_right(df):
  count = df.droite.value_counts()
  n_droite, n_gauche = count[True], count[False]
  df = df.sort_values(by = ["droite"], ascending = False)
  
  if n_droite > n_gauche :
    df = df[n_droite - n_gauche:]
  elif n_droite < n_gauche :
    df = df[2 * n_droite:]

  return df

df_brut = balance_left_right(df_brut)
df_brut.droite.value_counts()

True     51
False    51
Name: droite, dtype: int64

In [156]:
# Régler un problème de type
from ast import literal_eval
def convert_to_list(interventions):
  return literal_eval(str(interventions))

df_brut["interventions"] = df_brut["interventions"].apply(convert_to_list)
df_brut

,Unnamed: 0,groupe,nom,interventions,droite
68,68,LR,Jean-Marie Sermier,"[ Ce sont les départements qui payent ! , Pou...",True
71,71,LR,Laurence Trastour-Isnart,[ Il vise à supprimer la part salariale de la ...,True
66,66,LR,Marc Le Fur,"[Marc Le Fur Prochaine séance, demain, à neuf...",True
41,41,LR,Nicolas Forissier,[ Je ne vais pas reprendre les excellents argu...,True
39,39,LR,Jean-Jacques Gaultier,[ … mais contre votre gestion de la crise. Mai...,True
...,...,...,...,...,...
136,136,SOC,Marietta Karamanli,[ Je vous remercie de ces éléments et de votre...,False
137,137,SOC,Joël Aviragnet,"[ Il vise, lui aussi, à supprimer l'article in...",False
138,138,SOC,Boris Vallaud,[ Quelle incongruité que la majorité pénale so...,False
139,139,SOC,Alain David,[ Déposé par le groupe Socialistes et apparent...,False


In [161]:
# Séparer toutes les interventions en colonnes différentes
df_brut = df_brut.explode("interventions")

In [170]:
# Créer une feature "longeur de l'intervention"
df_brut = df_brut.assign(longeur = df_brut["interventions"].str.len())
df_brut

,Unnamed: 0,groupe,nom,interventions,droite,longeur
61,61,LR,Valérie Bazin-Malgras,Exactement ! Que faisons-nous donc ici ?,True,42
61,61,LR,Valérie Bazin-Malgras,Je rejoins les propos de mon collègue Larive ...,True,312
61,61,LR,Valérie Bazin-Malgras,Il a été rédigé par ma collègue Emmanuelle An...,True,200
60,60,LR,Nathalie Bassire,Les différents exposés de nos collègues ont b...,True,655
60,60,LR,Nathalie Bassire,Ce sous-amendement vise à compléter l'amendem...,True,651
...,...,...,...,...,...,...
156,156,SOC,George Pau-Langevin,"Avec cet amendement, nous voulons à nouveau e...",False,663
156,156,SOC,George Pau-Langevin,Nous appelons ici l'attention sur le fait que...,False,578
156,156,SOC,George Pau-Langevin,"Il vise à supprimer l'alinéa 9, qui rend poss...",False,660
156,156,SOC,George Pau-Langevin,Le groupe Socialistes et apparentés propose d...,False,421
